# Imports

In [1]:
from game import Game
from halma import Halma
import pandas as pd
import strategies
import heuristics
import time
from main import print_board
import numpy as np

# Strategies and heuristics

In [2]:
strategies_list = [
    strategies.RandomStrategy,
    strategies.BestNextStateStrategy, 
    lambda h: strategies.MinimaxStrategy(h, 2), 
    lambda h: strategies.AlphaBetaStrategy(h, 2)]

heuristics_list = [
    heuristics.euclides_state_heuristic,
    heuristics.manhattan_state_heuristic,
    heuristics.manhattan_euclides_state_heuristic,
    heuristics.heatmap_heuristic,
    heuristics.adaptive_heatmap_heuristic,
]

In [3]:
def collect_game_data(player_white_strategy, player_black_strategy, stop_threshold):
    game = Game(player_white_strategy, player_black_strategy)
    move_times = {player : [] for player in Halma.players}

    while not game.halma.finished and game.halma.round < stop_threshold:
        times_list = move_times[game.current_player]
        
        start = time.time()
        game.next_move()
        end = time.time()

        times_list.append(end - start)
    
    game_time = sum(sum(player_times) for player_times in move_times.values())

    return (game.halma.winner, game.halma.round, move_times, game_time, game)

# Random vs best_move

In [6]:

df = pd.DataFrame()

for heuristic in heuristics_list:
    winner, rounds, times, game_time, _ = collect_game_data(strategies.RandomStrategy(), strategies.BestNextStateStrategy(heuristic), 400)

    time_averages = {player : np.mean(np.array(times[player])) for player in Halma.players}

    df.loc[heuristic.__name__, 'Winner'] = None if winner is None else winner.name
    df.loc[heuristic.__name__, 'Rounds'] = rounds
    df.loc[heuristic.__name__, 'Game Time'] = game_time

    for player, avg_time in time_averages.items():
        df.loc[heuristic.__name__, f'{player.name}_AvgMoveTime'] = avg_time

df


,Winner,Rounds,Game Time,PLAYER_WHITE_AvgMoveTime,PLAYER_BLACK_AvgMoveTime
euclides_state_heuristic,PLAYER_BLACK,254.0,3.638662,0.000346,0.028305
manhattan_state_heuristic,PLAYER_BLACK,244.0,3.367154,0.000328,0.027272
manhattan_euclides_state_heuristic,PLAYER_BLACK,254.0,9.477334,0.000289,0.074335
heatmap_heuristic,PLAYER_BLACK,324.0,1.482220,0.000349,0.008801
adaptive_heatmap_heuristic,None,400.0,2.180062,0.000393,0.010560


# Heuristics comparison, best_move strategy

In [15]:
df = pd.DataFrame(index=pd.MultiIndex.from_arrays([[], []], names=[f"{player.name}_Heuristic" for player in Halma.players]))

for heuristic_1 in heuristics_list:
    for heuristic_2 in heuristics_list:
        winner, rounds, times, game_time, _ = collect_game_data(strategies.BestNextStateStrategy(heuristic_1), strategies.BestNextStateStrategy(heuristic_2), 400)

        time_averages = {player : np.mean(np.array(times[player])) for player in Halma.players}
        index = (heuristic_1.__name__, heuristic_2.__name__)
        df.loc[index, 'Winner'] = None if winner is None else winner.name
        df.loc[index, 'Rounds'] = rounds
        df.loc[index, 'Game Time'] = game_time

        for player, avg_time in time_averages.items():
            df.loc[index, f'{player.name}_AvgMoveTime'] = avg_time

df

Winner  \
PLAYER_WHITE_Heuristic             PLAYER_BLACK_Heuristic                             
euclides_state_heuristic           euclides_state_heuristic            PLAYER_BLACK   
                                   manhattan_state_heuristic                   None   
                                   manhattan_euclides_state_heuristic  PLAYER_BLACK   
                                   heatmap_heuristic                           None   
                                   adaptive_heatmap_heuristic                  None   
manhattan_state_heuristic          euclides_state_heuristic            PLAYER_WHITE   
                                   manhattan_state_heuristic           PLAYER_WHITE   
                                   manhattan_euclides_state_heuristic  PLAYER_WHITE   
                                   heatmap_heuristic                   PLAYER_WHITE   
                                   adaptive_heatmap_heuristic          PLAYER_WHITE   
manhattan_euclides_state_heuristic euclides_state_heuristic            PLAYER_BLACK   
                                   manhattan_state_heuristic                   None   
                                   manhattan_euclides_state_heuristic  PLAYER_BLACK   
                                   heatmap_heuristic                           None   
                                   adaptive_heatmap_heuristic                  None   
heatmap_heuristic                  euclides_state_heuristic                    None   
                                   manhattan_state_heuristic           PLAYER_WHITE   
                                   manhattan_euclides_state_heuristic          None   
                                   heatmap_heuristic                   PLAYER_WHITE   
                                   adaptive_heatmap_heuristic          PLAYER_WHITE   
adaptive_heatmap_heuristic         euclides_state_heuristic                    None   
                                   manhattan_state_heuristic           PLAYER_BLACK   
                                   manhattan_euclides_state_heuristic          None   
                                   heatmap_heuristic                   PLAYER_BLACK   
                                   adaptive_heatmap_heuristic                  None   

                                                                       Rounds  \
PLAYER_WHITE_Heuristic             PLAYER_BLACK_Heuristic                       
euclides_state_heuristic           euclides_state_heuristic             156.0   
                                   manhattan_state_heuristic            400.0   
                                   manhattan_euclides_state_heuristic   156.0   
                                   heatmap_heuristic                    400.0   
                                   adaptive_heatmap_heuristic           400.0   
manhattan_state_heuristic          euclides_state_heuristic             183.0   
                                   manhattan_state_heuristic            171.0   
                                   manhattan_euclides_state_heuristic   169.0   
                                   heatmap_heuristic                    199.0   
                                   adaptive_heatmap_heuristic           199.0   
manhattan_euclides_state_heuristic euclides_state_heuristic             156.0   
                                   manhattan_state_heuristic            400.0   
                                   manhattan_euclides_state_heuristic   156.0   
                                   heatmap_heuristic                    400.0   
                                   adaptive_heatmap_heuristic           400.0   
heatmap_heuristic                  euclides_state_heuristic             400.0   
                                   manhattan_state_heuristic            209.0   
                                   manhattan_euclides_state_heuristic   400.0   
                                   heatmap_heuristic                    179.0   
                          

# Best_move vs minimax depth = 2

In [16]:
df = pd.DataFrame()

for heuristic in heuristics_list:
    winner, rounds, times, game_time, _ = collect_game_data(strategies.BestNextStateStrategy(heuristic), strategies.MinimaxStrategy(heuristic, 2), 300)

    time_averages = {player : np.mean(np.array(times[player])) for player in Halma.players}

    df.loc[heuristic.__name__, 'Winner'] = None if winner is None else winner.name
    df.loc[heuristic.__name__, 'Rounds'] = rounds
    df.loc[heuristic.__name__, 'Game Time'] = game_time

    for player, avg_time in time_averages.items():
        df.loc[heuristic.__name__, f'{player.name}_AvgMoveTime'] = avg_time

    print(f"{heuristic.__name__} DONE!")
df

euclides_state_heuristic DONE!
manhattan_state_heuristic DONE!
manhattan_euclides_state_heuristic DONE!
heatmap_heuristic DONE!
adaptive_heatmap_heuristic DONE!


,Winner,Rounds,Game Time,PLAYER_WHITE_AvgMoveTime,PLAYER_BLACK_AvgMoveTime
euclides_state_heuristic,PLAYER_BLACK,172.0,645.044368,0.041633,7.458883
manhattan_state_heuristic,PLAYER_BLACK,214.0,768.295393,0.041691,7.138640
manhattan_euclides_state_heuristic,PLAYER_BLACK,172.0,1225.321883,0.080464,14.167464
heatmap_heuristic,PLAYER_WHITE,237.0,220.086801,0.011243,1.853804
adaptive_heatmap_heuristic,None,300.0,246.133919,0.010898,1.640935


# Best_move vs alpha_beta depth = 2

In [5]:
df = pd.DataFrame()

for heuristic in heuristics_list:
    winner, rounds, times, game_time, _ = collect_game_data(strategies.BestNextStateStrategy(heuristic), strategies.AlphaBetaStrategy(heuristic, 2), 300)

    time_averages = {player : np.mean(np.array(times[player])) for player in Halma.players}

    df.loc[heuristic.__name__, 'Winner'] = None if winner is None else winner.name
    df.loc[heuristic.__name__, 'Rounds'] = rounds
    df.loc[heuristic.__name__, 'Game Time'] = game_time

    for player, avg_time in time_averages.items():
        df.loc[heuristic.__name__, f'{player.name}_AvgMoveTime'] = avg_time

    print(f"{heuristic.__name__} DONE!")
df

euclides_state_heuristic DONE!
manhattan_state_heuristic DONE!
manhattan_euclides_state_heuristic DONE!
heatmap_heuristic DONE!
adaptive_heatmap_heuristic DONE!


,Winner,Rounds,Game Time,PLAYER_WHITE_AvgMoveTime,PLAYER_BLACK_AvgMoveTime
euclides_state_heuristic,PLAYER_BLACK,172.0,194.955740,0.035842,2.231086
manhattan_state_heuristic,PLAYER_BLACK,214.0,229.105206,0.030824,2.110346
manhattan_euclides_state_heuristic,PLAYER_BLACK,172.0,279.507546,0.079542,3.170546
heatmap_heuristic,PLAYER_WHITE,237.0,171.584682,0.011334,1.442677
adaptive_heatmap_heuristic,None,300.0,181.874745,0.010796,1.209768


# Heuristics comparison, alpha_beta depth = 2

In [18]:
df = pd.DataFrame(index=pd.MultiIndex.from_arrays([[], []], names=[f"{player.name}_Heuristic" for player in Halma.players]))

for heuristic_1 in heuristics_list:
    for heuristic_2 in heuristics_list:
        winner, rounds, times, game_time, _ = collect_game_data(strategies.AlphaBetaStrategy(heuristic_1, 2), strategies.AlphaBetaStrategy(heuristic_2, 2), 300)

        time_averages = {player : np.mean(np.array(times[player])) for player in Halma.players}
        index = (heuristic_1.__name__, heuristic_2.__name__)
        df.loc[index, 'Winner'] = None if winner is None else winner.name
        df.loc[index, 'Rounds'] = rounds
        df.loc[index, 'Game Time'] = game_time

        for player, avg_time in time_averages.items():
            df.loc[index, f'{player.name}_AvgMoveTime'] = avg_time

df

Winner  \
PLAYER_WHITE_Heuristic             PLAYER_BLACK_Heuristic                             
euclides_state_heuristic           euclides_state_heuristic            PLAYER_WHITE   
                                   manhattan_state_heuristic           PLAYER_WHITE   
                                   manhattan_euclides_state_heuristic  PLAYER_BLACK   
                                   heatmap_heuristic                   PLAYER_WHITE   
                                   adaptive_heatmap_heuristic          PLAYER_WHITE   
manhattan_state_heuristic          euclides_state_heuristic                    None   
                                   manhattan_state_heuristic                   None   
                                   manhattan_euclides_state_heuristic  PLAYER_BLACK   
                                   heatmap_heuristic                   PLAYER_WHITE   
                                   adaptive_heatmap_heuristic          PLAYER_WHITE   
manhattan_euclides_state_heuristic euclides_state_heuristic            PLAYER_WHITE   
                                   manhattan_state_heuristic                   None   
                                   manhattan_euclides_state_heuristic  PLAYER_BLACK   
                                   heatmap_heuristic                   PLAYER_WHITE   
                                   adaptive_heatmap_heuristic          PLAYER_WHITE   
heatmap_heuristic                  euclides_state_heuristic            PLAYER_BLACK   
                                   manhattan_state_heuristic                   None   
                                   manhattan_euclides_state_heuristic  PLAYER_BLACK   
                                   heatmap_heuristic                           None   
                                   adaptive_heatmap_heuristic                  None   
adaptive_heatmap_heuristic         euclides_state_heuristic            PLAYER_BLACK   
                                   manhattan_state_heuristic                   None   
                                   manhattan_euclides_state_heuristic  PLAYER_BLACK   
                                   heatmap_heuristic                           None   
                                   adaptive_heatmap_heuristic                  None   

                                                                       Rounds  \
PLAYER_WHITE_Heuristic             PLAYER_BLACK_Heuristic                       
euclides_state_heuristic           euclides_state_heuristic             215.0   
                                   manhattan_state_heuristic            275.0   
                                   manhattan_euclides_state_heuristic   208.0   
                                   heatmap_heuristic                    235.0   
                                   adaptive_heatmap_heuristic           235.0   
manhattan_state_heuristic          euclides_state_heuristic             300.0   
                                   manhattan_state_heuristic            300.0   
                                   manhattan_euclides_state_heuristic   254.0   
                                   heatmap_heuristic                    221.0   
                                   adaptive_heatmap_heuristic           221.0   
manhattan_euclides_state_heuristic euclides_state_heuristic             215.0   
                                   manhattan_state_heuristic            300.0   
                                   manhattan_euclides_state_heuristic   208.0   
                                   heatmap_heuristic                    217.0   
                                   adaptive_heatmap_heuristic           217.0   
heatmap_heuristic                  euclides_state_heuristic             202.0   
                                   manhattan_state_heuristic            300.0   
                                   manhattan_euclides_state_heuristic   176.0   
                                   heatmap_heuristic                    300.0   
                          